In [1]:
import os
from dotenv import load_dotenv
#from elasticsearch import Elasticsearch, ConnectionError

#es_client = Elasticsearch('http://localhost:9200')


# Load environment variables from the .env file
load_dotenv()
os.environ['ROOT_DATA_DIR']

'/Users/username/PycharmProjects/leaf-buddy/data'

In [2]:
# from IPython.display import clear_output

# !pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread
# clear_output()

# !rm gsheets.py
# !wget https://gist.githubusercontent.com/aleksandr-dzhumurat/c1b52a6284fab8e8355ef25cc3a3cc51/raw/2e0ef91fc111e7b838594dd01a309bd466dbafde/gsheets.py
# clear_output()
# print('Pakages installed')

In [3]:
# from gsheets import get_spreadsheet, duplicate_tab, insert_data_dummy

# new_tab_name = 'leafly_catalog'

# g_sheet_link = 'https://docs.google.com/spreadsheets/d/1cOZ930D8KVupQQciC_bUOXqrRlVytKhs5tXlDKjpLaY/edit?gid=0#gid=0'
# spreadsheet_id = g_sheet_link.split('/')[-2]
# print(spreadsheet_id)
# spreadsheet = get_spreadsheet(spreadsheet_id)
# duplicate_tab(spreadsheet, 'Sheet1', new_tab_name)

# worksheet = spreadsheet.worksheet(new_tab_name)

# insert_data_dummy(worksheet, search_results_df, ascending=[False, True])

In [109]:
import os
import pandas as pd

data_version = '7268cbe9f2b1'

leafly_full_catalog_index = None
leafly_full_catalog_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], f'leafly_catalog_{data_version}.csv'))
print(leafly_full_catalog_df.shape[0])
leafly_full_catalog_df.head(3)

49259


,doc_id,category,title,link,file,processed_file_name
0,doc_c2ccd832-edc6-4533-aa95-2b83997b6235,cannabis/flower,3.5g THC-A COOKIES PINK RUNTZ FLOWER HYBRID,/brands/aventus-8/products/aventus-8-torch-thc...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...
1,doc_0f4e3e88-e015-4358-a160-cf2b202dcf75,cannabis/flower,7.5 g THC-A Pink Rozay Hybrid,/brands/aventus-8/products/aventus-8-torch-thc...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...
2,doc_da5a1680-918e-4d80-b6cc-cd587edf90a1,cannabis/flower,LEMONADE BLANCO 1 G FLOWER THC,/brands/aventus-8/products/aventus-8-torch-thc...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...


In [6]:
sweed_catalog_df = None
if sweed_catalog_df is None:
    sweed_catalog_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], 'pipelines-data', 'products_dataset_ffb59da3.csv.gzip'))
print(f"Catalog length {sweed_catalog_df.shape[0]}, {sweed_catalog_df['ProductCategory'].unique()}")
sweed_catalog_df.head(3)                               

Catalog length 1513, ['Accessories Temp' 'Concentrates' 'Vapes' 'Apparel' 'Pre-Rolls' 'Flower'
 'Fees' 'Edibles' 'Oral Products' 'Accessories'
 'Accessories - Cabbage Club' 'Topicals' 'Temp Accessories'
 'Apparel - Cabbage Club']


,DealerID,ProductID,ProductName,ProductCategory,ProductCategoryID,ProductTypeID,ProductType,ProductBrand,ProductGroupName,ProductClass,ProductStrainPrevalence,ProductSubCategory,SizeName,ProductPrice,product_category,pre_size_name,size_name
0,211388,190856,Test Accessory EACH,Accessories Temp,3459,0,NaN,No Brand,Test Accessory,Non-cannabis,N/S,NaN,EACH,100,accessories temp,each,each
1,211388,263581,Grape Noir CDT Dablicator 1g,Concentrates,3483,0,NaN,MÜV,Grape Noir CDT Dablicator,Cannabis,Indica,CDT Dablicator,1g,60,concentrates,1g,1g
2,211388,263599,Layered Cake Badder 1g,Concentrates,3483,0,NaN,MÜV,Layered Cake Badder,Cannabis,Indica,Live Resin Badder,1g,60,concentrates,1g,1g


# [Leafly full catalog] -> [Sweed catalog] matching

dummy tf-idf search based on name + category

In [115]:
from IPython.display import display, HTML

from utils import TfidfRanker

def match_sweed_catalog_lo_leafly():
    ecom_catalog_df['ProductID'] = ecom_catalog_df[['name', 'category.name']].apply(lambda row: f'{row.iloc[0]} {row.iloc[1]}'.lower(), axis=1).apply(lambda x: get_match(x, index))
    ecom_catalog_df = ecom_catalog_df.merge(products_df[['ProductID', 'ProductName', 'ProductCategory']], on='ProductID')
    
    return ecom_catalog_df

if leafly_full_catalog_index is None:
    leafly_full_catalog_corpus = [
        (
            row['doc_id'],
            f"{row['title']} {row['category']}"
        )
        for rn, row in leafly_full_catalog_df.iterrows()
    ]
    leafly_full_catalog_index = TfidfRanker(leafly_full_catalog_corpus)
    print(leafly_full_catalog_index)

product_name, product_category, product_brand = sweed_catalog_df[sweed_catalog_df['ProductBrand'] != 'Sweet Supply'].sample(1)[['ProductName', 'ProductCategory', 'ProductBrand']].iloc[0]
print(f'Search results for: {product_name}, {product_brand}, {product_category}')
search_results = leafly_full_catalog_index.rank(product_name, top_n=5)
match_product_ids = [leafly_full_catalog_index.get_document(s[0]) for s in search_results]
    
base_url = 'https://www.leafly.com'
sub_df = leafly_full_catalog_df[leafly_full_catalog_df['doc_id'].isin(match_product_ids)][['category', 'title', 'link']]
for row in sub_df.to_dict(orient='records'):
    url = base_url+row['link']
    title = row['title']
    display(HTML(f'<a href="{url}" target="_blank">{title}</a>'))

Search results for: Miracle Fruit 0.5g, MÜV, Vapes


Brands available in catalog

In [112]:
sales_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], 'pipelines-data', 'sales_dataset_ffb59da3.csv.gzip'))

product_sales_df = (
    sales_df
    .groupby('ProductID')['CustomerID'].count()
    .reset_index(name='purchase_count')
    .sort_values(by='purchase_count', ascending=False)
)
brands_df = (
    product_sales_df
        .merge(sweed_catalog_df[['ProductID', 'ProductName', 'ProductCategory', 'ProductBrand']], on='ProductID')
        .groupby(['ProductBrand'])['ProductID'].count()
    .reset_index()
    .sort_values(by='ProductID', ascending=False)
)
brands_df.head()

,ProductBrand,ProductID
6,MÜV,439
12,Sweet Supply,130
0,(the) Essence,115
10,Savvy,75
14,Verano,26


# [Sweed catalog] -> [Leafly catalog] matching

using search

run script that download search pages using leafly catalog

```shell
SCENARIO=search_query make run-leafly-scraper
```

In [54]:
import os
import pandas as pd

def concat_search_results(leafly_search_results):
    matched_catalog = (
         pd.concat([
            pd.read_csv(os.path.join(leafly_search_results, i))
            for i in os.listdir(leafly_search_results) if '.csv' in i
        ])
     )
    search_results_df = (
        product_sales_df
        .merge(sweed_catalog_df[['ProductID', 'ProductName', 'ProductCategory', 'ProductBrand']], on='ProductID')
        .merge(matched_catalog.rename(columns={'sweed_product_id': 'ProductID', 'purchase_count': 'sweed_purchases'}), on='ProductID')
        .sort_values(by='purchase_count', ascending=False)
    )
    print(f'Num rows concatted {search_results_df.shape[0]}')
    search_results_df.to_csv(os.path.join('/Users/username/PycharmProjects/leaf-buddy/data/search', 'sweed_leafly_mapping.csv'), index=False)
    return search_results_df

data_version = 'e6da18c153cb'
leafly_search_results_dir = f'/Users/username/PycharmProjects/leaf-buddy/data/search/search_{data_version}'
search_results_df = concat_search_results(leafly_search_results_dir)
search_link_dedupled_df = search_results_df[['title', 'link']].groupby('title').agg(lambda x: x.iloc[0]).reset_index()

search_results_df.head(3)

10735


,ProductID,purchase_count,ProductName,ProductCategory,ProductBrand,sweed_product_name,title,doc_id,brand,link,item_url,price,uom
0,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE RSO | 10 PACK,doc_7efafe668209,Verano,/dispensary-info/cloud-nine-dispensary/p/63214...,https://www.leafly.com/dispensary-info/cloud-n...,$32.22,each
8,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],100mg Concord Grape RSO,doc_63481b638580,Verano,/dispensary-info/brute-s-roots/p/627740137/100...,https://www.leafly.com/dispensary-info/brute-s...,$29.00,each
1,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE - 10 PACK - RSO SOFT LOZENGES - ...,doc_9c516ef2fa58,Encore Edibles,/dispensary-info/elevated-herb-nj/p/637147171/...,https://www.leafly.com/dispensary-info/elevate...,$28.50,each


Post process search results using LLM as judge

In [57]:
import os

import pandas as pd

# from matching import prepare_matching


output_dir = os.path.join(os.environ['ROOT_DATA_DIR'], 'pipelines-data', 'matched_data')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
mapping_df = search_results_df[['ProductID', 'ProductName', 'ProductBrand', 'sweed_product_name', 'title', 'brand', 'doc_id']].drop_duplicates()
if len(os.listdir(output_dir)) == 0:
    # iterate by product ids and save
    prepare_matching(output_dir, mapping_df)
    print('Matching complited')
# concat search results (with matching algorithm applied)
final_df = pd.concat([pd.read_csv(os.path.join(output_dir, i)) for i in os.listdir(output_dir)])
app_file_output_path = os.path.join(os.environ['ROOT_DATA_DIR'], 'app_matched_leafly_sweed_dataset.csv')
cols = ['ProductID', 'ProductName', 'ProductBrand', 'title', 'brand', 'match_confidence']
ecom_catalog_prepared_df = (
    final_df[~final_df['match_confidence'].isna()][cols] # keep matched good only
    .merge(search_link_dedupled_df, on='title')
)
print(f'Num rows: {ecom_catalog_prepared_df.shape[0]}')
ecom_catalog_prepared_df.to_csv(app_file_output_path, index=False)

print(f'data saved to {app_file_output_path}')
ecom_catalog_prepared_df.head(3)

Num rows: 591
data saved to /Users/username/PycharmProjects/leaf-buddy/data/app_matched_leafly_sweed_dataset.csv


,ProductID,ProductName,ProductBrand,title,brand,match_confidence,link
0,193484,Mango Kush Dart 0.5g,Sweet Supply,"Mango Kush 0.5g, Live Rosin",TRENDI,high,/dispensary-info/zen-leaf-las-vegas-flamingo-r...
1,210053,G-Wagon Budder 1g,Sweet Supply,G-Wagon 3.5g,Victoria Cannabis Company,high,/dispensary-info/buds-smoke-cannabis---burling...
2,280332,Tropical Teeth #8 0.3g,(the) Essence,Tiramisu Teeth 3.5g,(the) Essence,high,/dispensary-info/green-rose-dispensary-river-n...


For matched items download html pages and extract plaintxt for an AI budtender

In [117]:
import os
from collections import defaultdict

import pandas as pd
from IPython.display import display, HTML
from tqdm import tqdm

from utils import (
    save_html, request_smart, create_scraper, url_to_filename,
    get_id_hash, process_item, prepare_dirs,
    get_id_hash
)
from utils import save_file, prepare_plaintext_files

from scrape_tools import dowlnload_leafly_items


data_version = '0ca973b7'

root_data_dir = os.environ['ROOT_DATA_DIR']
sweed_x_leafly_items_description_dir, html_items_dir, html_dir = prepare_dirs(root_data_dir, data_version, prefix='catalog_ecom_')
catalog_path = os.path.join(root_data_dir, f'leafly_ecom_catalog_{data_version}.csv')
if os.path.exists(catalog_path):
    catalog_df = pd.read_csv(catalog_path)
else:
    catalog_df = dowlnload_leafly_items(ecom_catalog_prepared_df, html_dir, sweed_x_leafly_items_description_dir)
    catalog_df.to_csv(catalog_path, index=False)
print(f"Num items in catalog: {catalog_df.shape[0]}")
sweed_x_leafly_files = os.listdir(sweed_x_leafly_items_description_dir)
if len(sweed_x_leafly_files) == 0:
    prepare_plaintext_files(catalog_df)
print(f'Plaintext files downloaded: {len(sweed_x_leafly_files)}')

Num items in catalog: 196
Plaintext files downloaded: 196


done! Plaintext data will help AI budtender

# [Sweed main page]

get sweed main page

In [20]:
from utils import read_json
from matching import prepare_main_page_catalog

json_catalog_file_name = 'get_product_carousel_list.json'
main_page_json = read_json('/Users/username/PycharmProjects/ml-research/data/pipelines_data', json_catalog_file_name)
ecom_main_page_catalog_df = prepare_main_page_catalog(main_page_json)
ecom_main_page_catalog_df.head(3)

(140, 26)
tags []
effects []
['Edibles' 'Flower' 'Vapes' 'Concentrates' 'Pre-Rolls' 'Topicals'
 'Oral Products' 'Apparel' 'Accessories']


,carousel_id,carousel_name,id,name,description,images,category.name,subcategory.name,brand.name,strain.name,strain.prevalence.id,strain.prevalence.name,strain.flavors,strain.terpenes,strain,product_image
0,3679,🔥Hot Deals🔥,141135,Concord Grape RSO Soft Chews,Each Encore RSO Soft Chew provides 10 mg of TH...,[https://media.sweedpos.com/store/production/1...,Edibles,RSO Soft Chews,Encore,Indica,4.0,Indica,[],[],NaN,https://media.sweedpos.com/store/production/17...
1,3679,🔥Hot Deals🔥,141416,Sour Black Cherry RSO Soft Chews,"The chewy chart-toppers. Sugary sativas, sweet...",[https://media.sweedpos.com/store/production/1...,Edibles,RSO Soft Chews,Encore,Hybrid,2.0,Hybrid,[],[],NaN,https://media.sweedpos.com/store/production/17...
2,3679,🔥Hot Deals🔥,141254,Mango RSO Soft Chews,Each Encore RSO Soft Chew provides 10 mg of TH...,[https://media.sweedpos.com/store/production/1...,Edibles,RSO Soft Chews,Encore,Sativa,6.0,Sativa,[],[],NaN,https://media.sweedpos.com/store/production/17...


In [99]:
def brand_stat(input_df, group_col, count_col, min_num_entries = None):
    brands_df = (
        input_df
        .groupby([group_col])[count_col].count()
        .reset_index()
        .sort_values(by=count_col, ascending=False)
    )
    if min_num_entries is not None:
        brands_df = brands_df.query(f'{count_col} > {min_num_entries}')
    return brands_df
sweed_x_leafly_bands_df = brand_stat(ecom_catalog_prepared_df, 'brand', 'ProductID', min_num_entries=3)
sweed_x_leafly_bands_df.head(10)

,brand,ProductID
86,Verano,308
0,(the) Essence,117
76,Savvy,36
67,Puffco,7
31,Encore Edibles,7
56,MJ Arsenal,5
6,Bits,5
83,TYSON 2.0,4


In [98]:
brand_stat(ecom_main_page_catalog_df, 'brand.name', 'id', min_num_entries=1)

,brand.name,id
4,MÜV,67
0,(the) Essence,17
3,Encore,16
6,Savvy,15
8,Sweet Supply,12
9,Verano,7
1,Bits,2
5,Puffco,2


# [Sweed main page] -> [Sweed catalog] matching

this step is only because we don't have actual product ids in main page

In [100]:
from matching import match_ecom_catalog_to_sweed_catalog
from utils import TfidfRanker

corpus = [
    (
        row['ProductID'],
        f"{row['ProductName']} {row['ProductBrand']} {row['ProductCategory']}"
    )
    for rn, row in sweed_catalog_df.iterrows()
]
sweed_index = TfidfRanker(corpus)
print(sweed_index)

matched_ecom_main_page_catalog_df = match_ecom_catalog_to_sweed_catalog(ecom_main_page_catalog_df, sweed_catalog_df, sweed_index)
cols = [
    i for i in matched_ecom_main_page_catalog_df.columns
    if i not in [
        'images', 'carousel_id', 'strain.prevalence.id', 'strain.terpenes', 'strain', 'strain.flavors'
    ]
]
data_version = '7268cbe9f2b1'
matched_catalog_path = os.path.join(os.environ['ROOT_DATA_DIR'], f'sweed_main_page_catalog_{data_version}.csv')
matched_ecom_main_page_catalog_df[cols].to_csv(matched_catalog_path, index=False)
print(f'Saved to {matched_catalog_path}, num rows {matched_ecom_main_page_catalog_df.shape[0]}')
matched_ecom_main_page_catalog_df[cols].head(3)

Saved to /Users/username/PycharmProjects/leaf-buddy/data/leafly_ecom_catalog_0ca973b7.csv, num rows 140


,carousel_name,id,name,description,category.name,subcategory.name,brand.name,strain.name,strain.prevalence.name,product_image,ProductID,ProductName,ProductCategory
0,🔥Hot Deals🔥,141135,Concord Grape RSO Soft Chews,Each Encore RSO Soft Chew provides 10 mg of TH...,Edibles,RSO Soft Chews,Encore,Indica,Indica,https://media.sweedpos.com/store/production/17...,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles
1,🔥Hot Deals🔥,141416,Sour Black Cherry RSO Soft Chews,"The chewy chart-toppers. Sugary sativas, sweet...",Edibles,RSO Soft Chews,Encore,Hybrid,Hybrid,https://media.sweedpos.com/store/production/17...,192987,Sour Black Cherry RSO Soft Chews 100mg [10-Pack],Edibles
2,🔥Hot Deals🔥,141254,Mango RSO Soft Chews,Each Encore RSO Soft Chew provides 10 mg of TH...,Edibles,RSO Soft Chews,Encore,Sativa,Sativa,https://media.sweedpos.com/store/production/17...,193165,Mango Soft Chews 100mg [10-Pack],Edibles


# AI budtender

Find relevant items on main page using the search index

In [121]:
ecom_catalog_prepared_df.shape[0], len(sweed_x_leafly_files)

(591, 196)

In [31]:
from utils import TfidfRanker

leafly_descriptions_corpus = [
    (
        row['doc_id'],
        open(row['processed_file_name'], 'r').read().replace('\n', ' ')
    )
    for _, row in catalog_df.iterrows()
]
print(f'{len(leafly_descriptions_corpus)}')
ecom_index = TfidfRanker(leafly_descriptions_corpus)
user_query = 'mint'
search_results = ecom_index.rank(user_query, top_n=2)

catalog_df[
    catalog_df['doc_id'].isin([ecom_index.get_document(i[0]) for i in search_results])
]

196


,doc_id,title,link,file,processed_file_name
150,doc_18cfc66aa7aa,Gelato 1g,/dispensary-info/sierra-wellness-connection---...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...
194,doc_f755e22ed5f5,Primal Animal Mint Crumble (H) 1g,/dispensary-info/nugz/p/639608736/primal-anima...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...


In [35]:
from IPython.display import display, HTML

base_url = 'https://www.leafly.com'

for i in search_results:
    sub_df = catalog_df[catalog_df['doc_id'] == ecom_index.get_document(i[0])].iloc[0]
    url = base_url+sub_df['link']
    title = sub_df['title']
    display(HTML(f'<a href="{url}" target="_blank">{title}</a>'))
    print(leafly_descriptions_corpus[i[0]][1])
    print('------')

Primal Animal Mint Crumble (H) 1g, Concentrate - 
------


This Bay Area-native gets its name from its fruity, dessert-like aroma. Gelato buds have distinct, fiery orange hairs and a shining white coat of crystal resin. Novices should approach this powerhouse with caution, but those with a high tolerance will delight in its heavy-handed euphoria.  -- Lineage: Sherbet x Thin Mint Cookies -- Type: Indica -- Effects: Euphoric, Relaxing
------


In [101]:
# search_results_df[
#     search_results_df['ProductID'] == search_results_df[search_results_df['title'] == 'Primal Animal Mint Crumble (H) 1g']['ProductID'].item()
# ]

In [111]:
# sweed_catalog_df[sweed_catalog_df['ProductName']==random_name]
# print(sweed_catalog_df.shape[0])
# sweed_catalog_df[sweed_catalog_df['ProductName'].apply(lambda x: 'slurricane' in x.lower())]

In [86]:
verano_data_version = 'ee72f7351d78'
verano_catalog_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], f'verano_leafly_catalog_{verano_data_version}.csv'))
print(verano_catalog_df.shape[0])
verano_catalog_df[verano_catalog_df['title'].apply(lambda x: 'layered' in x.lower())]

404


,doc_id,category,title,link,file,processed_file_name


In [74]:
random_name = sweed_catalog_df[sweed_catalog_df['ProductCategory']== 'Pre-Rolls'].sort_values('ProductName')['ProductName'].sample(1).iloc[0]
print(random_name)
#catalog_df.iloc[[i[0] for i in index.query(random_name)[:5]]]

Punch Cookies 3.5g [5-Pack]


In [47]:
from time import sleep
from IPython.display import clear_output

def monitor_scraping(data_version):
    prev = None
    while True:
        num_files = len(os.listdir(f'/Users/username/PycharmProjects/leaf-buddy/data/leafly_html_pages_{data_version}/items_html_pages_{data_version}'))
        if prev is None:
            prev = num_files
        else:
            if num_files == prev:
                raise RuntimeError(f'Scrapping was stucked, num files: {num_files}')
            prev = num_files
        print(f'num files {num_files}')
        sleep(10)
        clear_output()


        
        # monitor_scraping(data_version='7268cbe9f2b1')

In [52]:
import requests

from IPython.display import display, HTML

api_url = 'http://0.0.0.0:8000'

def request_api(query, relief, positive_effects):
    headers = {"Content-Type": "application/json"}
    payload = {
        "relief": relief,
        "positive_effects": positive_effects,
        "query": query
    }
    print('sending request...')
    response = requests.post(
        f"{api_url}/search",
        headers=headers,
        json=payload
    )
    resp = [
        (i['title'], i['url']) for i in response.json()
    ]
    return resp

# 'insomnia'
for title, url in request('good sleep', [], []):
    display(HTML(f'<a href="{url}" target="_blank">{title}</a>'))

# Keyword search

In [244]:
def elastic_search(es_client, query, limit=3, filter=None, random=False):
    index_name = 'greenbro-content'
    if random:
        search_query = {
          "size": 10,
          "query": {
            "function_score": {
              "functions": [
                {
                  "random_score": {}
                }
              ]
            }
          }
        }
    else:
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["relief", "positive_effects", "flavours"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {  # TODO: add filter as parameter
                        "term": {
                            "category": "flower"
                        }
                    }
                }
            }
        }
    response = es_client.search(index=index_name, body=search_query)
    
    return response['hits']['hits'][:limit]

def pretty(search_results):
    result_docs = []

    include_fields = ['title', 'tags', 'relief', 'positive_effects', 'flavours']
    for hit in search_results:
        result_docs.append({k: v for k, v in  hit['_source'].items() if k in include_fields})
    return result_docs

try:
    query = 'Lemon, headache'
    
    res = elastic_search(es_client, query, random=True)
    sample = res[0]['_source']
    # print(sample.keys())
    print()
    print(pretty(res))
except ConnectionError:
    print('Elastic unreachable')


[{'title': 'OG 18', 'tags': 'Indica Dominant|70% Indica/30% Sativa', 'relief': 'arthritis chronic pain depression fibromyalgia headaches inflammation insomnia migraines', 'positive_effects': 'euphoria happy', 'flavours': 'diesel pine sweet'}, {'title': 'Peanut Butter Breath', 'tags': 'Hybrid,|50%/50%', 'relief': 'chronic pain depression loss of appetite nausea stress', 'positive_effects': 'body high hungry relaxing uplifting', 'flavours': 'herbal pine woody'}, {'title': 'Purple Sunset', 'tags': 'Indica Dominant|60% Indica/40% Sativa', 'relief': 'add/adhd anxiety depression stress', 'positive_effects': 'calming creative hungry relaxing uplifting', 'flavours': 'citrus fruity orange sour spicy sweet'}]


# Hybrid search

In [ ]:
import requests

def get_embed(query):
    url = "http://localhost:8000/embed"
    data = {"text": query}
    response = requests.post(url, json=data)
    if response.status_code == 200:
        res = np.array(json.loads(response.json()['embed']))
    else:
        print("Error:", response.status_code, response.text)
        raise RuntimeError
    return res

def elastic_hybrid_search(es_client, query, limit=3, filter=None, random=False):
    index_name = 'greenbro-content'
    vector_search_term = get_embed(query)
    knn_query = {
        "field": "text_vector",
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 100
    }
    
    response = es_client.search(
        index=index_name,
        query={
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["relief", "positive_effects", "flavours"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {  # TODO: add filter as parameter
                        "term": {
                            "category": "flower"
                        }
                    }
                }
        },
        knn=knn_query,
        size=5
    )
    return response['hits']['hits'][:limit]


try:
    query = 'Lemon, headache'
    
    res = elastic_hybrid_search(es_client, query, random=True)
    sample = res[0]['_source']
    #print(sample.keys())
    #print()
    print(pretty(res))
except ConnectionError:
    print('Elastic unreachable')

In [251]:
[{'score': i['_score'], 'res': i['_source']['item_name']} for i in res]

[{'score': 0.7898849, 'res': 'lemon-tree'},
 {'score': 0.77863276, 'res': 'golden-lemons'},
 {'score': 0.7349596, 'res': 'lemonhead'}]

In [ ]:


[{'score': scores[i], 'doc': self.documents[i]} for i in idx]

# Prepare data

In [211]:
import os
import pandas as pd

current_file_dir = os.getcwd()
root_data_dir = os.path.join(os.path.dirname(current_file_dir), 'data')
app_files_dir = os.path.join(root_data_dir, 'pipelines-data')

print(os.listdir(app_files_dir))

res_csv_path = os.path.join(app_files_dir, 'content_green_bro.csv')

content_db = pd.read_csv(res_csv_path)

['content_queries.json', 'golden_dataset.json', 'content_reviews_green_bro.json', 'models', 'api_db.csv', 'content_green_bro.csv', 'content_db_df.csv', 'ground_truth.json']


In [4]:
content_db.head()

,title,item_name,tags,relief,positive_effects,flavours,avg_rating,num_ratings
0,Casey Jones,casey-jones,Sativa Dominant|80% Sativa/20% Indica,add/adhd arthritis bipolar disorder chronic pa...,creative euphoria focus happy,citrus fruity lemon sweet,4.6,38
1,Gorilla Glue #4,gorilla-glue-4,Indica Dominant|60% Indica/40% Sativa,add/adhd bipolar disorder chronic pain depress...,body high euphoria happy uplifting,chemical chocolate coffee diesel pine sweet,4.6,339
2,Cherry Pie,cherry-pie,Indica Dominant|80% Indica/20% Sativa,add/adhd anxiety bipolar disorder chronic pain...,creative euphoria happy relaxing,berry cherry sweet,4.5,110
3,Amnesia Haze,amnesia-haze,Sativa Dominant|80% Sativa/20% Indica,add/adhd anxiety cancer chronic pain depressio...,creative euphoria giggly happy,citrus lemon sweet,4.5,55
4,Blueberry,blueberry,Indica Dominant|80% Indica/20% Sativa,add/adhd bipolar disorder depression insomnia ...,euphoria happy sleepy,berry blueberry sweet woody,4.5,95


Data for service

In [189]:
def add_url(postfix):
    url = os.path.join('https://www.allbud.com' + content_db_full.iloc[0]['link'])

content_db_full = pd.read_csv(os.path.join(app_files_dir, 'content_db_df.csv'))
content_db_full['url'] = content_db_full['link'].apply(lambda x: 'https://www.allbud.com' + x)
content_db = content_db_full[['url', 'item_name']].merge(content_db, on='item_name')

content_db.to_csv(os.path.join(app_files_dir, 'api_db.csv'), index=False)

In [192]:
def get_candidates(content_names_list):
    res = [
        {
            'title': row['title'], 'url': row['url'],
            'explanation': f"{row['tags']}: {row['positive_effects']}",
            'flavours': row['flavours']
        }
        for _, row in content_db[content_db['item_name'].isin(content_names_list)].iterrows()
    ]
    return res

content_names_list = ['blueberry', 'amnesia-haze', 'cherry-pie']

candidates = get_candidates(content_names_list)

candidates

[{'title': 'Cherry Pie',
  'url': 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/cherry-pie',
  'explanation': 'Indica Dominant|80% Indica/20% Sativa: creative euphoria happy relaxing',
  'flavours': 'berry cherry sweet'},
 {'title': 'Amnesia Haze',
  'url': 'https://www.allbud.com/marijuana-strains/sativa-dominant-hybrid/amnesia-haze',
  'explanation': 'Sativa Dominant|80% Sativa/20% Indica: creative euphoria giggly happy',
  'flavours': 'citrus lemon sweet'},
 {'title': 'Blueberry',
  'url': 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/blueberry',
  'explanation': 'Indica Dominant|80% Indica/20% Sativa: euphoria happy sleepy',
  'flavours': 'berry blueberry sweet woody'}]

In [204]:
os.listdir(app_files_dir)

['content_queries.json',
 'golden_dataset.json',
 'content_reviews_green_bro.json',
 'models',
 'api_db.csv',
 'content_green_bro.csv',
 'content_db_df.csv',
 'ground_truth.json']

In [217]:
models_dir = os.path.join(root_data_dir, 'pipelines-data', 'models')
index_file_path = os.path.join(models_dir, 'embeds_index.json')
embeds_file_path = os.path.join(models_dir, 'embeds.npy')
with open(index_file_path, 'r') as f:
    index = json.load(f)
embeds = np.load(embeds_file_path)
db = {}
for i, embed in enumerate(embeds):
    db[index[i]] = embed
print(db[index[-1]])

In [ ]:
class VectorDB:
    def __init__(self, index, embeddings):
        self.db = {}
        for i, embed in enumerate(embeddings):
            self.db[index[i]] = embed

    def get_item_vector(self, item_name):
        return self.db[item_name]


def get_vector_db(root_dir):
    models_dir = os.path.join(root_data_dir, 'pipelines-data', 'models')
    index_file_path = os.path.join(models_dir, 'embeds_index.json')
    embeds_file_path = os.path.join(models_dir, 'embeds.npy')
    with open(index_file_path, 'r') as f:
        index = json.load(f)
    embeds = np.load(embeds_file_path)
    print(embeds.shape, len(index))
    vector_db = VectorDB(index=index, embeddings=embeds)
    return vector_db

vector_db = get_vector_db(current_file_dir)
#vector_db.get_item_vector('gorilla-glue-4')

In [221]:
vector_db.db

{}

In [209]:
os.listdir('/Users/username/PycharmProjects/leaf-bro/jupyter_notebooks/data/pipelines-data/models')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/username/PycharmProjects/leaf-bro/jupyter_notebooks/data/pipelines-data/models'

In [201]:
import json
import datetime
import hashlib
import random

import numpy as np
import backoff
import openai
from openai import OpenAI


client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)


@backoff.on_exception(backoff.expo, openai.APIError)
@backoff.on_exception(backoff.expo, openai.RateLimitError)
@backoff.on_exception(backoff.expo,openai.Timeout)
@backoff.on_exception(backoff.expo, RuntimeError)
def gpt_query(gpt_params, verbose: bool = False, avoid_fuckup: bool = False) -> dict:
    print('connecting OpenAI...')
    if verbose:
        print(gpt_params["messages"][1]["content"])
    response = client.chat.completions.create(
        **gpt_params
    )
    gpt_response = response.choices[0].message.content
    if avoid_fuckup:
        if '[' in gpt_response or '?' in gpt_response or '{' in gpt_response:
            raise RuntimeError
    res = {'recs': gpt_response}
    res.update({'prompt_tokens': response.usage.completion_tokens, 'prompt_tokens': response.usage.prompt_tokens, 'total_tokens': response.usage.total_tokens})
    seed_phrase = f'{str(datetime.datetime.now().timestamp())}{gpt_response}'
    generation_id = str(hashlib.md5(seed_phrase.encode('utf-8')).hexdigest())[:12]
    res.update({'id': generation_id})
    return res

def random_shuffle(input_list):
    """
    my_list = ['apple', 'banana', 'cherry', 'date', 'fig']
    random_shuffle(my_list)
    """
    res = random.shuffle(input_list)
    return ', '.join(input_list)

def promt_generation(candidates):
  # TODO: use jinja2
  flavours_list = ['sweet', 'citrus', 'fruity', 'spicy', 'berry', 'pine', 'herbal', 'sour', 'lemon', 'woody', 'grape', 'tropical', 'diesel', 'skunky', 'nutty', 'blueberry', 'vanilla', 'creamy', 'candy', 'cherry']
  reliefs_list = ['depression', 'chronic pain', 'stress', 'insomnia', 'anxiety', 'fatigue', 'nausea', 'migraines', 'muscle spasms', 'headaches', 'ptsd', 'loss of appetite', 'add/adhd', 'inflammation', 'arthritis', 'bipolar disorder', 'cramps', 'pms', 'fibromyalgia', 'gastrointestinal disorder']
  promt = f"""
      generate 5 search query based on strain description below. All queries should be 3-5 words long. Do not include strain name.
      Do not add word "strain".
      Base query on flavours (not connstrained with this list): {random_shuffle(flavours_list)}
      reliefs: {random_shuffle(reliefs_list)}
      effects (body and mind)
      {candidates}
      Queries:
  """
  return promt

def promt_generation_2(candidates, query):
  promt = f"""
      Below you can find items with description in format `title: description`
      {candidates}
      Rerank items and return reranked item ids base on user query. Return only reranked items, comma-separate
      Do not add any explanation, just result
      User query: {query}
      expected result: [title, title, title]
      reranked:
  """
  return promt

def generate(gpt_prompt, verbose=False):
    gpt_params = {
        'model': 'gpt-3.5-turbo',
        'max_tokens': 500,
        'temperature': 0.7,
        'top_p': 0.5,
        'frequency_penalty': 0.5,
    }
    # gpt_promt = promt_generation(gen_candadates(db, ids))
    if verbose:
        print(gpt_promt)
    messages = [
        {
          "role": "system",
          "content": "You are a helpful assistant for medicine shopping",
        },
        {
          "role": "user",
          "content": gpt_prompt,
        },
    ]
    gpt_params.update({'messages': messages})
    res = gpt_query(gpt_params, verbose=False)
    return res


def load_reviews(reviews_file_path):
    with open(reviews_file_path, 'r') as f:
        reviews_dict = json.load(f)
    return reviews_dict

def get_item_reviews(item_name, reviews_dict):
    reviews = [i['review'].replace('\n', ' ') for i in reviews_dict if i['item_name'] == item_name]
    return reviews

def aggregate_reviews(content_df, reviews_dict):
    res = {}
    for name in content_df['item_name'].unique():
      res[name] = {'reviews': ''.join(get_item_reviews(name, reviews_dict))}
    return res

def test_generation(reviews_agregated, random_strain):
    prompt = promt_generation(reviews_agregated[random_strain]['reviews'])
    generated_result = generate(prompt)
    print(generated_result['recs'])
    print()
    reviews_agregated[random_strain]['reviews']

def test_reranking(candidates, query):
    [f"{i['title']}: {i['explanation']}, {i['flavours']}" for i in candidates]
    prompt = promt_generation_2(candidates, query)
    generated_result = generate(prompt)
    print(generated_result['recs'])

# reviews_dict = load_reviews(os.path.join(app_files_dir, 'content_reviews_green_bro.json'))
# reviews_agregated = aggregate_reviews(content_db, reviews_dict)
# # print(reviews_agregated['fire-og']['reviews'])

# random_strain = np.random.choice(list(reviews_agregated.keys()))
# test_generation(reviews_agregated, random_strain)

test_reranking(candidates, 'avoid cherry')

connecting OpenAI...
Blueberry, Amnesia Haze


In [203]:
a = []
a[0]

IndexError: list index out of range

Search index

In [71]:
key = list(reviews_agregated.keys())[0]
print(key)
reviews_agregated[key]

casey-jones


{'reviews': " First off, an elevation of my mood, a little bit of rejuvenation to my body, and even my thoughts, I noticed quickly while smoking CJ. I was feeling a little sluggish before I smoked this strain. No more. The taste has some earthiness to it, but mostly that sweet citrus taste. If you're fond of sativas I would recommend trying CJ. Smoke this pot.  I got an eighth of Casey Jones and it kicks some serious backside. I felt very refreshed taking a puff of this light green, fuzzy bud and got notes of tangerine, citrus and even a bit of watermelon. I highly recommend Casey Jones as a daytime strain to inspire creativity and socialization.  I didn't have the pleasure of growing her, but I bought a half oz of 'b' buds and squished it. What a sweet strain. Sweet as in sugar sweet. It came out of the press as foam mostly it was so sugary. Then I burned it 1 time and it turned Into beautiful thick amber plastic. For a low-level thc strain, I was very satisfied. Ty  This strain is ph

Generate ground trurh

In [66]:
from IPython.display import clear_output

def generate_and_save_ground_truth(input_dict, output_filename):
    for i in input_dict:
      reviews = input_dict[i]['reviews']
      prompt = promt_generation(reviews)
      input_dict[i]['queries'] = generate(prompt)
    with open(output_filename, 'w') as f:
      json.dump(input_dict, f)
    print('Data generated')
    return input_dict

clear_output()
output_filename = os.path.join(app_files_dir,'content_queries.json')
user_queries_dict = generate_and_save_ground_truth(reviews_agregated, output_filename)
user_queries_dict[random_strain]

{'reviews': " My first time with the Chapo, was a multi-high experience the flavor, omg…. The throat on a few hits reminded me of the drainage you’d get from decent quality “coke”, very sweet head high, spurts of clear thought, energy, then my favorite “Couch Lock”, and the world for now ceases.  Smoked a pre-roll of el chapo OG and fucking hell my eyes got sooo heavy and got baked asf!  I have a chronic disease this indica strain is a cure all fo me and I’m a long time smoker!!  I really enjoy this strains ability to reduce pain.  El Chapo OG Kush (Face/Off OG x SFV OG x OG Kush) by Boss Status Genetics/Cali Buds.Average Cannabinoid Content: THC: 10-23%CBD: 0.00-0.50%Terpene Profile:Myrcene, Limonene, Pinene, beta-Caryophyllene Medicinal Benefits include:pain relief gastrointestinal issues nausea relief insomnia relief relaxation appetite stimulationdepression relief  Superb! The best strain I've had in 2019 except that Presidential Kush. Grade A!  El Chapo OG (AKA: Shorty) is an Indi

In [ ]:

output_file_path = os.path.join(app_files_dir,'ground_truth.json')

In [68]:
#queries_dict = load_reviews(output_filename)
random_strain = np.random.choice(list(reviews_agregated.keys()))

print(queries_dict[random_strain]['queries']['recs'])

1. Fruity pain relief options
2. Energizing headache remedy
3. Relaxing stress relief choices
4. Uplifting mood support picks
5. Cerebral anxiety solutions


In [54]:
import re

def clean_text(text):
    cleaned_text = re.sub(r'^\d+\.\s*', '', text)
    return cleaned_text

def prepare_ground_truth(queries_dict, data_path):
    ground_truth = []
    
    for k in queries_dict:
        queries = queries_dict[k]['queries']['recs'].split('\n')
        ground_truth += [{'answer': k, 'query': clean_text(q)} for q in queries]
    print('Num generates: %d' % len(ground_truth))
    with open(data_path, 'w') as f:
      json.dump(ground_truth, f)
    print('data saved to %s' % data_path)
    return ground_truth
output_file_path = os.path.join(app_files_dir,'ground_truth.json')
ground_truth = prepare_ground_truth(queries_dict, output_file_path)

Num generates: 1455
data saved to /Users/username/PycharmProjects/leaf-bro/data/pipelines-data/ground_truth.json


In [56]:
import numpy as np

random_id = np.random.randint(0, len(ground_truth))
print(ground_truth[random_id])

queries_dict[ground_truth[random_id]['answer']]['reviews']

{'answer': 'gorilla-cookies', 'query': 'Citrus herbal pain relief'}


" Great high that is good for a night downtown that lasts for hours  This one hits me more like an Indica than a Sativa despite it being Sativa leaning. I get the heavy couch-locking feeling of the Gorilla Glue with the euphoric aspect (mentally and physically) from the GSC, along with that nice head buzz. While remaining clear-headed, I do find myself struggling to keep my eyes open after a good few dabs.  Awesome strain I was bouncing around getting things done Happy and care free Pretty dumb to be honest but didnt care  After consuming gorilla cookies thc oil in a vape device, numerous times, as a medical & recently recreational user, gorilla cookies is an extremely high sativa dominant smoke that in my case gives me plenty of nervous energy but no focus to get things done! Slight couch lock that's easily overcome for most, I personally didn't like the restless energy & feelings of anxiety gorilla cookies always produces. Your experience may differ!I didn't feel any other benefits s

# Project

find a strain and motivate user to buy it

In [76]:
def prepare_catalog(reviews_agregated):
    index = []
    corpus = []
    for key, content in reviews_agregated.items():
        index.append(key)
        corpus.append(content['reviews'])
    return index, corpus
index, corpus = prepare_catalog(reviews_agregated)
print(len(index), len(corpus))

291 291


In [87]:
models_dir = os.path.join(app_files_dir, 'models')

index_file_path = os.path.join(models_dir, 'embeds_index.json')
embeds_file_path = os.path.join(models_dir, 'embeds.npy')
with open(index_file_path, 'r') as f:
    index = json.load(f)
embeds = np.load(embeds_file_path)
print(embeds.shape, len(index))

(291, 768) 291


In [88]:
def get_pytorch_model(models_dir, model_name='multi-qa-distilbert-cos-v1'):
  from sentence_transformers import SentenceTransformer

  model_path = os.path.join(models_dir, model_name)

  if not os.path.exists(model_path):
      print('huggingface model loading...')
      embedder = SentenceTransformer(model_name)
      embedder.save(model_path)
  else:
      print('pretrained model loading...')
      embedder = SentenceTransformer(model_name_or_path=model_path)
  print('model loadind done')

  return embedder

class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [{'score': scores[i], 'doc': self.documents[i]} for i in idx]

embedder = get_pytorch_model(models_dir)
embedder

/Users/username/.pyenv/versions/3.10.13/envs/llmops-env/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


pretrained model loading...
model loadind done


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [89]:
search_engine = VectorSearchEngine(documents=index, embeddings=embeds)


In [91]:
user_question = 'citrus relaxation'

v = embedder.encode(user_question)

search_engine.search(v, num_results=5)

[{'score': 0.4901401, 'doc': 'lemon-tree'},
 {'score': 0.47702503, 'doc': 'golden-lemons'},
 {'score': 0.47451043, 'doc': 'hardcore-og'},
 {'score': 0.45465428, 'doc': 'orange-crush'},
 {'score': 0.4432276, 'doc': 'skywalker-og'}]

In [129]:
random_numbers = np.random.choice(np.arange(0, len(ground_truth) - 1), size=100, replace=False)
res = []
for random_id in random_numbers:
    origin = ground_truth[random_id]['answer']
    query = ground_truth[random_id]['query']
    v = embedder.encode(query)
    res.append( 1 if sum(origin==i['doc'] for i in search_engine.search(v, num_results=30)) > 0 else 0)

print(sum(res), len(res))

39 100


In [131]:
[ground_truth[i] for i in random_numbers[np.where(res==1)]]


/var/folders/6_/c5005cfj5_s0k2v7t69q1qsh0000gn/T/ipykernel_39617/649090524.py:1: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  [ground_truth[i] for i in random_numbers[np.where(res==1)]]


[]

In [166]:
def eval_hitrate(golden_dataset):
    res = []
    for entry in golden_dataset:
        v = embedder.encode(entry['query'])
        res.append( 1 if sum(entry['origin']==i['doc'] for i in search_engine.search(v, num_results=30)) > 0 else 0)
    print('Hit rate: %.3f, num entries: %d' % (sum(res)/len(res), len(res)))

def generate_golden_set():
    items_list = list(set([i['answer'] for i in ground_truth]))
    random.shuffle(items_list)
    random.shuffle(ground_truth)
    result = []
    test_queries = []
    for i in items_list[:100]:
        result.append(i)
        test_queries.append([j['query'] for j in ground_truth if j['answer']==i][0])
    golden_dataset = [{'origin': origin, 'query': query} for origin, query in zip(result, test_queries)]
    return golden_dataset

data_path = os.path.join(app_files_dir, 'golden_dataset.json')
golden_dataset = [{'origin': origin, 'query': query} for origin, query in zip(result, test_queries)]
with open(data_path, 'w') as f:
  json.dump(golden_dataset, f)

In [160]:
[i['answer'] for i in np.array(ground_truth)[np.where(np.array(res)==1)]]

['alaskan-thunder-fuck',
 'key-lime-pie',
 'la-confidential',
 'grape-pie',
 'monster-cookies',
 'white-og',
 'purple-punch',
 'island-sweet-skunk',
 'gobbstopper',
 'gelato-cake',
 'white-widow',
 'chocolate-diesel',
 'lavender-jones',
 'sour-berry',
 '101-headband',
 'super-boof',
 'sfv-og',
 'cherry-pie',
 '101-headband',
 'chocolate-diesel',
 'kosher-kush',
 'granddaddy-purple',
 'master-kush',
 'zoap',
 'papaya-cake',
 'tahoe-og',
 'sour-tangie',
 'lemon-skunk',
 'slurricane',
 'zkittlez-cake']

In [163]:
# queries_dict['skywalker-og']['reviews']